In [1]:
import pandas as pd

### Eerst gaan we alle datafiles die we nodig hebben mergen tot 1 dataframe

#### 1. Alles van Account mergen

##### Account mergen met activiteit tabellen

In [2]:
data_acc = pd.read_csv('../data_clean/Account_fixed.csv', sep=',')
data_acti = pd.read_csv('../data_clean/Activiteitscode_fixed.csv', sep=",")
data_acc_acti = pd.read_csv('../data_clean/Account_activiteitscode_fixed.csv', sep=",")

data_acc = data_acc[data_acc['account_reden_van_status'] != 'Inactief']
data_acc = data_acc[data_acc['account_reden_van_status'] != 'Stopzetting']
data_acc = data_acc[data_acc['account_adres_provincie'] == 'Oost-Vlaanderen']

# merge account_activiteitscode with activiteitscode
merged_relation = data_acc_acti.merge(data_acti, left_on='account_activiteitscode_activiteitscode', right_on='activiteitscode_activiteitscode_id', how='inner')
# merge account with the merged relation from above
accounts_merged = data_acc.merge(merged_relation, left_on='account_account_id', right_on='account_activiteitscode_account', how='inner')

accounts_merged = accounts_merged[accounts_merged['activiteitscode_status'] == 'Actief']

In [3]:
# RAM vrijmaken
try:
    del data_acc
    del data_acti
    del data_acc_acti
    del merged_relation
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [4]:
accounts_merged.shape

(3192, 22)

##### Accounts mergen met financiële data

In [5]:
financiele_data = pd.read_csv('../data_clean/Account_financiële_data_fixed.csv', sep=',')
financiele_data.shape

(2030180, 6)

In [6]:
accounts_merged = accounts_merged.merge(financiele_data, left_on='account_account_id', right_on='financieledata_ondernemingid', how='inner')
accounts_merged.shape

(44590, 28)

##### Accounts merge met Contactfiches

In [7]:
contactfiches = pd.read_csv('../data_clean/Contact_fixed.csv', sep=',')
contactfiches = contactfiches[contactfiches['contact_status'] != 'Inactief']
contactfiches['contact_functietitel'] = contactfiches['contact_functietitel'].str.lower().str.replace(r'[^\w\s]', '', regex=True) \
                                                        .str.replace('  ', ' ').str.strip()
contactfiches.shape

(393514, 6)

In [8]:
accounts_merged = accounts_merged.merge(contactfiches, left_on='account_account_id', right_on='contact_account', how='inner')
accounts_merged.shape

(244928, 34)

##### Accounts mergen met Functie

In [9]:
contactfiches_functies = pd.read_csv('../data_clean/Contact_functie_fixed.csv', sep=',')
functies = pd.read_csv('../data_clean/Functie_fixed.csv', sep=',')
contactfiches_functies.shape, functies.shape

((589129, 2), (71, 2))

In [10]:
contactfiches_functies.columns

Index(['contactfunctie_contactpersoon', 'contactfunctie_functie'], dtype='object')

In [11]:
accounts_merged = accounts_merged.merge(contactfiches_functies, left_on='contact_contactpersoon_id', right_on='contactfunctie_contactpersoon', how='inner')

In [12]:
list(accounts_merged.columns)

['account_account_id',
 'account_adres_geografische_regio',
 'account_adres_geografische_subregio',
 'account_adres_plaats',
 'account_adres_postcode',
 'account_adres_provincie',
 'account_industriezone_naam_',
 'account_is_voka_entiteit',
 'account_ondernemingsaard',
 'account_ondernemingstype',
 'account_oprichtingsdatum',
 'account_primaire_activiteit',
 'account_reden_van_status',
 'account_status',
 'account_voka_nr_',
 'account_adres_land',
 'account_activiteitscode_account',
 'account_activiteitscode_activiteitscode',
 'account_activiteitscode_inf_account_inf_activiteitscodeid',
 'activiteitscode_naam',
 'activiteitscode_activiteitscode_id',
 'activiteitscode_status',
 'financieledata_ondernemingid',
 'financieledata_boekjaar',
 'financieledata_aantal_maanden',
 'financieledata_toegevoegde_waarde',
 'financieledata_fte',
 'financieledata_gewijzigd_op',
 'contact_contactpersoon_id',
 'contact_account',
 'contact_functietitel',
 'contact_persoon_id',
 'contact_status',
 'contact_

In [13]:
accounts_merged = accounts_merged.merge(functies, left_on='contactfunctie_functie', right_on='functie_functie_id', how='inner')

##### Accounts merge met Personen

In [14]:
# personen = pd.read_csv('../data_clean/Persoon_fixed.csv', sep=',')
# personen.shape

In [15]:
# merged_accounts = merged_accounts.merge(personen, left_on='contact_persoon_id', right_on='persoon_persoon_id', how='inner')
# merged_accounts.shape

#### 2. Alles van Afspraken mergen

In [16]:
# RAM vrijmaken
try:
    del financiele_data
    del contactfiches
    del contactfiches_functies
    del functies
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [17]:
afspraak_betreft_account_cleaned = pd.read_csv('../data_clean/Afspraak_betreft_account_cleaned_fixed.csv', sep=',')
afspraak_account_gelinkt_cleaned = pd.read_csv('../data_clean/Afspraak_account_gelinkt_cleaned_fixed.csv', sep=',')
afspraak_betreft_contact_cleaned = pd.read_csv('../data_clean/Afspraak_betreft_contact_cleaned_fixed.csv', sep=',')

In [18]:
afspraak_betreft_account_cleaned.shape, afspraak_account_gelinkt_cleaned.shape, afspraak_betreft_contact_cleaned.shape

((4876, 7), (2934, 7), (2552, 7))

In [19]:
afspraak_betreft_account_cleaned = afspraak_betreft_account_cleaned.drop(['afspraak_betreft_account_betreft_id'], axis=1)
afspraak_account_gelinkt_cleaned = afspraak_account_gelinkt_cleaned.drop(['afspraak_account_gelinkt_account'], axis=1)
afspraak_betreft_contact_cleaned = afspraak_betreft_contact_cleaned.drop(['afspraak_betreft_contactfiche_betreft_id'], axis=1)

In [20]:
afspraak_betreft_account_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']
afspraak_account_gelinkt_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']
afspraak_betreft_contact_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']

In [21]:
afspraak_betreft_account_cleaned["afspraak_betreft"] = "account"
afspraak_account_gelinkt_cleaned["afspraak_betreft"] = "account"
afspraak_betreft_contact_cleaned["afspraak_betreft"] = "contact"

In [22]:
afspraken_merged = pd.concat([afspraak_betreft_account_cleaned, afspraak_account_gelinkt_cleaned, afspraak_betreft_contact_cleaned])

In [23]:
afspraken = pd.read_csv('../data_clean/Afspraak_alle_fixed.csv', sep=',')
afspraken.columns = ['afspraak_afspraak_id']
afspraken.head()

,afspraak_afspraak_id
0,00082DAA-24BF-ED11-83FF-6045BD8952CE
1,00109C77-4D09-EB11-8114-001DD8B72B61
2,0011A5D9-CC9A-EC11-B400-0022488005A7
3,0016532E-14E3-EC11-BB3C-000D3ABCA38B
4,003760C1-13B4-EC11-983F-00224883C880


In [24]:
afspraken_merged = pd.merge(afspraken, afspraken_merged, on='afspraak_afspraak_id', how='inner')
afspraken_merged.reset_index(drop=True, inplace=True)

In [25]:
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10362 entries, 0 to 10361
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   afspraak_afspraak_id  10362 non-null  object
 1   afspraak_thema        10362 non-null  object
 2   afspraak_subthema     10362 non-null  object
 3   afspraak_onderwerp    10362 non-null  object
 4   afspraak_eindtijd     10362 non-null  object
 5   afspraak_keyphrases   10362 non-null  object
 6   afspraak_betreft      10362 non-null  object
dtypes: object(7)
memory usage: 566.8+ KB


In [26]:
# RAM vrijmaken
try:
    del afspraak_betreft_account_cleaned
    del afspraak_account_gelinkt_cleaned
    del afspraak_betreft_contact_cleaned
    del afspraken
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [27]:
afspraken_merged = afspraken_merged.drop_duplicates(subset=['afspraak_afspraak_id'], keep='first')
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7624 entries, 0 to 10360
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   afspraak_afspraak_id  7624 non-null   object
 1   afspraak_thema        7624 non-null   object
 2   afspraak_subthema     7624 non-null   object
 3   afspraak_onderwerp    7624 non-null   object
 4   afspraak_eindtijd     7624 non-null   object
 5   afspraak_keyphrases   7624 non-null   object
 6   afspraak_betreft      7624 non-null   object
dtypes: object(7)
memory usage: 476.5+ KB


In [28]:
activiteit_vereist_contact = pd.read_csv('../data_clean/Activiteit_vereist_contact_fixed.csv', sep=',')
activiteit_vereist_contact.shape

(4635, 2)

In [29]:
afspraken_merged = afspraken_merged.merge(activiteit_vereist_contact, left_on='afspraak_afspraak_id', right_on='activiteitvereistcontact_activityid_id', how='left')
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7754 entries, 0 to 7753
Data columns (total 9 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   afspraak_afspraak_id                    7754 non-null   object
 1   afspraak_thema                          7754 non-null   object
 2   afspraak_subthema                       7754 non-null   object
 3   afspraak_onderwerp                      7754 non-null   object
 4   afspraak_eindtijd                       7754 non-null   object
 5   afspraak_keyphrases                     7754 non-null   object
 6   afspraak_betreft                        7754 non-null   object
 7   activiteitvereistcontact_activityid_id  4545 non-null   object
 8   activiteitvereistcontact_reqattendee    4545 non-null   object
dtypes: object(9)
memory usage: 545.3+ KB


#### 3. Alles van Campagnes mergen

##### Campagne mergen met Sessie

In [30]:
campagne = pd.read_csv('../data_clean/Campagne_fixed.csv', sep=',')
campagne.shape

(4092, 11)

In [31]:
sessie = pd.read_csv('../data_clean/Sessie_fixed.csv', sep=',')
sessie.shape

(9403, 8)

In [32]:
result = (sessie.assign(sessie_thema_naam_list=sessie['sessie_thema_naam_'].str.split(', '))
               .explode('sessie_thema_naam_list')
               .drop_duplicates()
               .groupby('sessie_campagne')['sessie_thema_naam_list']
               .agg(lambda x: list(set(x)))
               .reset_index()
               .sort_values(by='sessie_thema_naam_list', key=lambda x: x.str.len(), ascending=False))
result

,sessie_campagne,sessie_thema_naam_list
3153,CBEE5437-9928-EB11-8117-001DD8B72B62,"[Marketing & Sales, IT & Technologie, Opvolgin..."
976,416A235F-F209-E911-80FA-001DD8B72B62,"[IT & Technologie, Strategie, Human Resources,..."
487,210CE445-B4A3-ED11-AAD1-6045BD895D85,"[IT & Technologie, Duurzaam Ondernemen, Human ..."
420,1CA4BA0E-6DA7-EB11-811B-001DD8B72B61,"[Marketing & Sales, IT & Technologie, Strategi..."
3738,F244649C-50BC-EA11-8113-001DD8B72B62,"[Marketing & Sales, Strategie, Duurzaam Ondern..."
...,...,...
1350,58F3F1ED-2A32-EC11-8127-001DD8B72B62,[Communicatie]
1351,58F9E91D-F350-E211-9B51-005056B06EC4,[unknown]
1352,58FA767E-ABB8-E111-A45C-00505680000A,[unknown]
1353,59337839-771C-E411-956D-005056B06EB4,[Internationaal Ondernemen]


Er zijn meerdere thema's in sessie per campagne, we gaan een nieuwe kolom maken in campagne die een lijst bevat van alle voorkomende thema's per campagne. Later gaan we deze multi hot encoden.

In [33]:
campagnes_merged = campagne.merge(result, left_on='campagne_campagne_id', right_on='sessie_campagne', how='inner')
campagnes_merged.shape

(3951, 13)

#### 4. Accounts mergen met Afspraken

In [34]:
accounts_merged = accounts_merged.merge(afspraken_merged, left_on='contact_contactpersoon_id', right_on='activiteitvereistcontact_reqattendee', how='inner')
accounts_merged.shape

(57157, 47)

In [35]:
# RAM vrijmaken
try:
    del afspraken_merged
    del activiteit_vereist_contact
    del result
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

#### 5. Cleaning voor de grote merge

In [36]:
onnodige_kolommen_accounts = [
 'account_account_id',
 'account_adres_postcode',
 'account_industriezone_naam_',
 'account_oprichtingsdatum',
 'account_adres_plaats',
 'account_voka_nr_',
 'account_is_voka_entiteit',
 'account_status',
 'account_adres_geografische_regio',
 'account_adres_provincie',
 'account_reden_van_status',
 'account_adres_land',
 'account_activiteitscode_account',
 'account_activiteitscode_activiteitscode',
 'account_activiteitscode_inf_account_inf_activiteitscodeid',
 'activiteitscode_activiteitscode_id',
 'contact_account',
 'contact_functietitel',
 'contact_persoon_id',
 'contact_voka_medewerker',
 'contact_status',
 'afspraak_afspraak_id', 
 'afspraak_onderwerp',
 'afspraak_eindtijd',
 'financieledata_ondernemingid',
 'financieledata_gewijzigd_op',
 'activiteitvereistcontact_activityid_id',
 'activiteitvereistcontact_reqattendee',
 'contactfunctie_contactpersoon',
 'contactfunctie_functie',
 'functie_functie_id',
 'activiteitscode_status'
]

onnodige_kolommen_campagnes = [
 'campagne_campagne_nr',
 'campagne_einddatum',
 'campagne_naam',
 'campagne_reden_van_status',
 'campagne_naam_in_email',
 'campagne_startdatum',
 'campagne_url_voka_be',
 'campagne_status',
 'sessie_campagne',
]

In [37]:
accounts_merged = accounts_merged.drop(onnodige_kolommen_accounts, axis=1)
campagnes_merged = campagnes_merged.drop(onnodige_kolommen_campagnes, axis=1)

In [38]:
list(accounts_merged.columns)

['account_adres_geografische_subregio',
 'account_is_voka_entiteit',
 'account_ondernemingsaard',
 'account_ondernemingstype',
 'account_primaire_activiteit',
 'activiteitscode_naam',
 'financieledata_boekjaar',
 'financieledata_aantal_maanden',
 'financieledata_toegevoegde_waarde',
 'financieledata_fte',
 'contact_contactpersoon_id',
 'functie_naam',
 'afspraak_thema',
 'afspraak_subthema',
 'afspraak_keyphrases',
 'afspraak_betreft']

In [39]:
list(campagnes_merged.columns)

['campagne_campagne_id',
 'campagne_type_campagne',
 'campagne_soort_campagne',
 'sessie_thema_naam_list']

#### 6. Alles mergen tot 1 dataframe met een cross join

In [40]:
# campagnes_merged['cross_join_key'] = 1
# accounts_merged['cross_join_key'] = 1

In [41]:
# merged_total = accounts_merged.merge(campagnes_merged, on='cross_join_key', how='outer').drop('cross_join_key', axis=1)

In [42]:
# # RAM vrijmaken
# try:
#     del accounts_merged
#     del campagnes_merged
# except:
#     print("Data is niet meer beschikbaar / is al verwijderd")

#### 7. Target label genereren

In [43]:
import pandas as pd
inschrijving = pd.read_csv('../data_clean/Inschrijving_fixed.csv', sep=',')
inschrijving.shape

(49175, 8)

In [44]:
merged_total["ingeschreven"] = 0

for index, row in merged_total.iterrows():
    if ((inschrijving['inschrijving_contactfiche'] == row['contact_contactpersoon_id']) & (inschrijving['inschrijving_campagne'] == row['campagne_campagne_id'])).any():
        merged_total.at[index, 'ingeschreven'] = 1

In [45]:
# merged_total["ingeschreven"].value_counts()

#### 8. Verdere data cleaning en One/Multi hot encoding

In [46]:
# merged_total = merged_total.drop(['contact_contactpersoon_id', 'campagne_campagne_id'], axis=1)

In [47]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer

oneHot = OneHotEncoder(sparse=False)
multiHot = MultiLabelBinarizer()

In [51]:
subregio_1hot = oneHot.fit_transform(accounts_merged[['account_adres_geografische_subregio']])

C:\Users\marlo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [58]:
def transform_role(role):
    if "Verantwoordelijke" in role:
        return "Verantwoordelijke"
    elif "Medewerker" in role:
        return "Medewerker"
    else:
        return role

accounts_merged['functie_naam'] = accounts_merged['functie_naam'].apply(transform_role)

In [ ]:
functie_1hot = oneHot.fit_transform(accounts_merged[['functie_naam']])

In [63]:
diensten_column = []
productie_column = []

for label in accounts_merged["account_ondernemingsaard"]:
    if label == "Diensten":
        diensten_column.append(1)
        productie_column.append(0)
    elif label == "Productie":
        diensten_column.append(0)
        productie_column.append(1)
    elif label == "Productie & Diensten":
        diensten_column.append(1)
        productie_column.append(1)
    elif label == "unknown":
        diensten_column.append(0)
        productie_column.append(0)

ondernemingsaard_multihot = pd.DataFrame({"Diensten": diensten_column, "Productie": productie_column})

In [ ]:
ondernemingstype_1hot = oneHot.fit_transform(accounts_merged[['account_ondernemingstype']])

In [71]:
accounts_merged["afspraak_betreft"].value_counts()

afspraak_betreft
account    52095
contact     5062
Name: count, dtype: int64

In [ ]:
afspraak_betreft = oneHot.fit_transform(accounts_merged[['afspraak_betreft']])